In [112]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import os

import torch
from PIL import Image
from torch import nn
from tqdm import tqdm

from models.style_transfer import SoundStyleTransferModel

In [113]:
model = SoundStyleTransferModel()

An error occurred while trying to fetch riffusion/riffusion-model-v1: riffusion/riffusion-model-v1 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch riffusion/riffusion-model-v1: riffusion/riffusion-model-v1 does not appear to have a file named diffusion_pytorch_model.safetensors.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


Loaded Traced UNet


## Train TVE

In [114]:
def get_audio_dataloader(batch_size=1, dataset="church_bell"):
    images = []
    path = f"./audios/timbre/{dataset}"
    for filename in os.listdir(path):
        if filename.endswith(".png"):
            image = Image.open(os.path.join(path, filename))
            image = SoundStyleTransferModel.preprocess_image(image)
            image = image.squeeze(0)
            images.append(image)

    class MyDataset(torch.utils.data.Dataset):
        def __init__(self, dataset, label):
            self.dataset = dataset
            self.label = label

        def __len__(self):
            return len(self.dataset)

        def __getitem__(self, idx):
            return self.dataset[idx], self.label

    dataloader = torch.utils.data.DataLoader(MyDataset(images, dataset), batch_size=batch_size, shuffle=True)
    return dataloader

In [115]:
# learning_rate = 0.001
# batch_size = 1
#
# for param in model.text_transform.text_encoder.parameters():
#     param.requires_grad = False
#
# for param in model.unet.parameters():
#     param.requires_grad = False
#
# optimizer = torch.optim.AdamW(model.text_transform.tve.parameters(), lr=learning_rate, weight_decay=0.0001)
# criterion = nn.MSELoss()
# scaler = torch.amp.GradScaler("cuda")
#
# dataset = "cornet"
# dataloader = get_audio_dataloader(batch_size=batch_size, dataset=dataset)
#
# num_epochs = 2000
# tqdm_bar = tqdm(range(num_epochs))
# model.text_transform.tve.train()
# for epoch in tqdm_bar:
#     epoch_loss = 0
#     for step, (images, labels) in enumerate(dataloader):
#         images, labels = images.to(device=model.device), ["*"] * batch_size
#
#         with torch.no_grad():
#             init_latents = model.encode_images(images)
#
#         noise = torch.randn_like(init_latents)
#         timesteps = torch.randint(
#             0,
#             model.scheduler.config.num_train_timesteps,
#             (batch_size,),
#             dtype=torch.int64,
#             device=model.device
#         )
#         noisy_latents = model.scheduler.add_noise(init_latents, noise, timesteps)
#
#         with torch.no_grad():
#             text_embeddings = [model.text_transform.embed_text(label) for label in labels]
#             text_embeddings = torch.stack(text_embeddings).squeeze(dim=1).to(device=model.device)
#
#         label_embeddings = model.text_transform.tve(timesteps, text_embeddings)
#
#         with torch.amp.autocast("cuda"):
#             pred_noise = model(noisy_latents, label_embeddings, timesteps)
#             loss = criterion(noise, pred_noise)
#             epoch_loss += loss.item()
#
#         optimizer.zero_grad()
#         scaler.scale(loss).backward()
#         scaler.step(optimizer)
#         scaler.update()
#
#     tqdm_bar.set_description(f"Loss: {loss.item() / len(dataloader)}")
#
# torch.save(model.text_transform.tve.state_dict(), "./data/tve.pth")

## Sample sound

In [118]:
model.text_transform.tve.load_state_dict(torch.load("./data/tve.pth", weights_only=True))
image = Image.open("sample.png")
print(image.size)
#image = Image.open("./audios/timbre/cornet/data1.png")

prompt = "*"
image = model.pipe_paper(
    image,
    prompt,
    inference_steps=50,
    scale=3.5,
    strength=1,
    alpha=0.5,
    tve=True
)
#image = model.transfer_style(image, "cornet", "church bell")

image.save("out_sample.png")
print(image.size)

(502, 512)
cuda
torch.Size([1, 3, 480, 512])


  0%|          | 0/50 [00:00<?, ?it/s]


RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript, serialized code (most recent call last):
  File "code/__torch__/diffusers/models/unet_2d_condition/___torch_mangle_4939.py", line 53, in forward
    _19 = (mid_block).forward(_17, _5, input, )
    _21 = (_0).forward(_19, _17, _5, _18, _14, )
    _22 = (_1).forward(_21, _15, _5, input, _16, _11, )
           ~~~~~~~~~~~ <--- HERE
    _23 = (_2).forward(_22, _12, _5, input, _13, _7, )
    _24 = (_3).forward(_23, _8, _5, input, _9, _6, )
  File "code/__torch__/diffusers/models/unet_2d_blocks/___torch_mangle_4649.py", line 30, in forward
    resnets1 = self.resnets
    _01 = getattr(resnets1, "0")
    input0 = torch.cat([argument_1, argument_2], 1)
             ~~~~~~~~~ <--- HERE
    _3 = (_00).forward((_01).forward(input0, argument_3, ), input, )
    input1 = torch.cat([_3, argument_5], 1)

Traceback of TorchScript, original code (most recent call last):
/home/ubuntu/.local/lib/python3.